# Проект для «Викишоп» с BERT

Целью работы является построение модели, которая могла бы классифицировать комментарии пользователей интернет-магазина "Викишоп" на позитивные и негативные.

Заказчик предоставил исходные данные, в которых имеется разметка о токсичности комментариев.

Качество модели необходимо будет измерять метрикой **F1**, ее результат на тестовой выборке должен быть не меньше `0,75`

**Описание данных:**

Данные находятся в файле toxic_comments.csv:

- **text** - текст комментария
- **toxic** — целевой признак

<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Импорт-библиотек-и-подготовка-данных" data-toc-modified-id="Импорт-библиотек-и-подготовка-данных-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Импорт библиотек и подготовка данных</a></span></li><li><span><a href="#Обучение-моделей" data-toc-modified-id="Обучение-моделей-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Обучение моделей</a></span><ul class="toc-item"><li><span><a href="#TF-IDF-+-LogisticRegression" data-toc-modified-id="TF-IDF-+-LogisticRegression-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>TF-IDF + LogisticRegression</a></span></li><li><span><a href="#fastText" data-toc-modified-id="fastText-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>fastText</a></span></li><li><span><a href="#BERT" data-toc-modified-id="BERT-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>BERT</a></span></li></ul></li><li><span><a href="#Проверка-на-тестовой-выборке" data-toc-modified-id="Проверка-на-тестовой-выборке-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Проверка на тестовой выборке</a></span></li><li><span><a href="#Общий-вывод-по-работе" data-toc-modified-id="Общий-вывод-по-работе-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Общий вывод по работе</a></span></li></ul></div>

## Импорт библиотек и подготовка данных

In [8]:
!pip install tqdm

In [3]:
# импортируем необходимые библиотеки
import numpy as np
import pandas as pd
import re


from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from tqdm import tqdm
from scipy.sparse import hstack

import warnings
warnings.filterwarnings('ignore')

In [ ]:
try:
    import fasttext
except:
    !pip install fasttext
    import fasttext

In [ ]:
!pip install -U spacy
!python3 -m spacy download en_core_web_sm

In [4]:
import spacy

In [5]:
try:
    import torch
except:
    !pip install torch
    import torch

In [4]:
try:
    from transformers import TrainingArguments, Trainer
    from transformers import BertTokenizer, BertForSequenceClassification, TextClassificationPipeline
except:
    !pip install transformers
    from transformers import TrainingArguments, Trainer
    from transformers import BertTokenizer, BertForSequenceClassification, TextClassificationPipeline

In [6]:
# если можно, открываем файл из локального носителя
# или с Jupyter Hub
try:
    df = pd.read_csv('toxic_comments.csv')
except:
    df = pd.read_csv('/datasets/toxic_comments.csv')

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159292 entries, 0 to 159291
Data columns (total 3 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   Unnamed: 0  159292 non-null  int64 
 1   text        159292 non-null  object
 2   toxic       159292 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 3.6+ MB


In [8]:
df['toxic'].value_counts()

0    143106
1     16186
Name: toxic, dtype: int64

In [9]:
df['text'].head(10)

0    Explanation\nWhy the edits made under my usern...
1    D'aww! He matches this background colour I'm s...
2    Hey man, I'm really not trying to edit war. It...
3    "\nMore\nI can't make any real suggestions on ...
4    You, sir, are my hero. Any chance you remember...
5    "\n\nCongratulations from me as well, use the ...
6         COCKSUCKER BEFORE YOU PISS AROUND ON MY WORK
7    Your vandalism to the Matt Shirvington article...
8    Sorry if the word 'nonsense' was offensive to ...
9    alignment on this subject and which are contra...
Name: text, dtype: object

Имеем 159 тысяч строк, на английском языке. В тексте встречаются знаки препинания, разный регистр букв, а также символы переноса текста (`\n`). Пропусков не имеется.

Явно выражен дисбаланс классов: токсичных комментариев только 10% от всего числа данных. При делении на выборки будем использовать параметр `stratify`.

Сразу выделим 20% данных на проведение теста модели, полученной в ходе этой работы.

In [10]:
train, test = train_test_split(df, test_size=0.2, random_state=42, stratify=df['toxic'])
train, test = train.reset_index(drop=True), test.reset_index(drop=True)

С помощью регулярных выражений очистим текст от всех символов, кроме слов, пробелов и апострофов. Приведем к нижнему регистру:

In [11]:
def clear_text(x):
    
    '''Функция, принимает на вход текст, и возвращает текст согласно шаблону, приведенный к нижнему регистру'''
    
    tx = ' '.join((re.sub(r'[^\w\s\']',' ',x).split())).lower()
    return tx

In [12]:
train['lemma_text'] = train['text'].apply(clear_text)

In [13]:
test['lemma_text'] = test['text'].apply(clear_text)

In [14]:
train['lemma_text']


0         it's been nearly two months and you still have...
1         i'm withdrawing my support i do not support wi...
2         what is this all about the day before yesterda...
3         a demon possessed pedophile pedophile alone wa...
4         you are abusing your position as admin to trol...
                                ...                        
127428    well so far there is no such article you can m...
127429    2012 utc you clean up after me when i self nom...
127430    question it seems that ip 98 179 149 193 has b...
127431    seattle biomed again did you read my message t...
127432    i started a discussion thread hi thanks for re...
Name: lemma_text, Length: 127433, dtype: object

In [ ]:
# Если можно, загружаем файл с лемматизированным текстом
# Если файла нет, приведем к лемме (занимает 12 минут)

try:
    train = pd.read_csv('train_with_lemm.csv', encoding='utf-8')
except:
    text_raw = train['lemma_text'].values
    lemmed_final = []
    nlp = spacy.load('en_core_web_sm')
    for text in nlp.pipe(text_raw,  disable=['parser','ner'], n_process=-1):
        lemmed_final.append(' '.join([token.lemma_ for token in text]))
    train['lemma_text'] = pd.Series(lemmed_final)
    train.to_csv('train_with_lemm.csv')



In [19]:
train_lemm = train

**Промежуточный вывод**:

На этом этапе мы рассмотрели исходные данные, очистили текст от символов, привели к нижнему регистру. Также отложили тестовую выборку на конечную оценку качества модели. После лемматизировали тренировочный текст.

## Обучение моделей

При планировании работы я решил воспользоваться TF-IDF с последующим применением логистической регрессии, моделью fastText (более новая и точная версия библиотеки векторного представления языка чем Word2Vec) и моделью BERT с тонкой настройкой параметров.

BERT буду обучать на графическом процессоре на удаленном сервере. После загружу модель в тетрадь с кодом настройки.

### TF-IDF + LogisticRegression

Признаками сделаем векторы слов и векторы словосочетаний каждого текста. Сначала проведем векторизацию, после с помощью `hstack` соберем признаки. Также при векторизации слов передадим стоп-слова.

In [20]:
word_vectorizer = TfidfVectorizer(
    sublinear_tf=True,
    analyzer='word',
    stop_words='english',
    ngram_range=(1, 1),
    max_features=10000)

In [21]:
char_vectorizer = TfidfVectorizer(
    sublinear_tf=True,
    analyzer='char',
    ngram_range=(2, 2),
    max_features=50000)


Для промежуточной оценки качества выделим валидационную выборку.

In [22]:
features_train, features_valid, target_train, target_valid = train_test_split(
    train['lemma_text'], train['toxic'], test_size=0.1, random_state=42, stratify=train['toxic'])
features_train, features_valid = features_train.values.astype('str'), features_valid.values.astype('str')

In [23]:
word_vectorizer.fit(features_train)
train_word_features = word_vectorizer.transform(features_train)
valid_word_features = word_vectorizer.transform(features_valid)

char_vectorizer.fit(features_train)
train_char_features = char_vectorizer.transform(features_train)
valid_char_features = char_vectorizer.transform(features_valid)

In [24]:
train_features = hstack([train_char_features, train_word_features])
valid_features = hstack([valid_char_features, valid_word_features])

In [25]:
# создаем модель логистической регрессии
clf = LogisticRegression(C=5, solver='sag')

In [26]:
clf.fit(train_features, target_train)

LogisticRegression(C=5, solver='sag')

In [27]:
# настраиваем отсечку, максимизируя метрику F1

predict = clf.predict_proba(valid_features)[:,1]
best_thres = 0.01
max_f_score = 0
for i in range(1,100):
    thres = i / 100
    var = f1_score(target_valid, list(map(int, predict >= thres)))
    if var > max_f_score:
        max_f_score = var
        best_thres = thres
best_thres

0.34

In [28]:
print('F1 = {}'.format(max_f_score))

F1 = 0.7940823670531787


**Оценка**:

Неплохое качество, выдвинутого заказчиком порога мы уже добились. Из минусов данной модели - плохо работает с новыми словами в данных.

### fastText

Хорошая библиотека, но признаки в нее подаются определнным способом. Подготовим их.

Во многих туториалах используют "сырой" текст. Поэкспериментировав на наших данных, пришел к выводу, что лемматизированные данные выдают более высокое качество. Их и будем использовать.

In [29]:
train = train[['toxic','lemma_text']]
train

,toxic,lemma_text
0,0,it be be nearly two month and you still have n...
1,0,I be withdraw my support I do not support wiki...
2,0,what be this all about the day before yesterda...
3,1,a demon possess pedophile pedophile alone be n...
4,0,you be abuse your position as admin to troll a...
...,...,...
127428,0,well so far there be no such article you can m...
127429,0,2012 utc you clean up after I when I self nomi...
127430,0,question it seem that ip 98 179 149 193 have b...
127431,0,seattle biome again do you read my message to ...


In [30]:
# для промежуточной оценки качества выделим валидационную выборку.
train, valid = train_test_split(
    train, test_size=0.1, random_state=42, stratify=train['toxic'])

In [31]:
valid

,toxic,lemma_text
73699,0,and 216 37 216 42 warn
109288,0,thank I will contact you via your talk page he...
73742,0,your word you will retract your comment from m...
99434,1,thank you hei it be I the william hope fan gue...
14653,1,be you still a dirty jew be you accept jesus a...
...,...,...
28554,0,post 2001 bike it be unclear whether the gentl...
114388,0,what policy be I not go by be there some rule ...
123950,0,dyk do you know be update on 24 february 2007 ...
1794,0,talkpage please do not edit my talkpage


Добавим префикс в наш таргет (так требует fastText).
Для дальнейшего сохранения в обучающий файл выделим все данные в один столбец.
Для этого напишем функцию:

In [32]:
def create_data_ft(df):
    df = df[['toxic','lemma_text']]
    df['true'] = df['toxic']
    df['toxic'] = '__class__' + df['toxic'].astype('str') + ' '
    df['toxic_lemma_text'] = df['toxic'] + df['lemma_text']
    return df

In [33]:
train = create_data_ft(train)
valid = create_data_ft(valid)

Для дальнейшего сохранения в обучающий файл выделим все данные в один столбец:

In [34]:
train.head(5)

,toxic,lemma_text,true,toxic_lemma_text
116933,__class__0,do side effect section add,0,__class__0 do side effect section add
65013,__class__0,photo i d file cucurbita_2011_g1 jpg in that p...,0,__class__0 photo i d file cucurbita_2011_g1 jp...
48249,__class__0,agree that the second half be opinion but shou...,0,__class__0 agree that the second half be opini...
36054,__class__0,thank thank for all the work you do at wikiped...,0,__class__0 thank thank for all the work you do...
93949,__class__0,be there someone who can stop koncorde from ce...,0,__class__0 be there someone who can stop konco...


In [35]:
valid.head(5)

,toxic,lemma_text,true,toxic_lemma_text
73699,__class__0,and 216 37 216 42 warn,0,__class__0 and 216 37 216 42 warn
109288,__class__0,thank I will contact you via your talk page he...,0,__class__0 thank I will contact you via your t...
73742,__class__0,your word you will retract your comment from m...,0,__class__0 your word you will retract your com...
99434,__class__1,thank you hei it be I the william hope fan gue...,1,__class__1 thank you hei it be I the william h...
14653,__class__1,be you still a dirty jew be you accept jesus a...,1,__class__1 be you still a dirty jew be you acc...


In [36]:
# сохраняем наши данные в файл, чтобы в последствии "накормить" ими fastText

train.to_csv('fastText.train', header=None, index=False, columns=['toxic_lemma_text'],encoding='utf-8') 
valid.to_csv('fastText.valid', header=None, index=False, columns=['toxic_lemma_text'],encoding='utf-8') 

In [37]:
model_ft = fasttext.train_supervised(input='fastText.train', label="__class__", lr=1, epoch=10, loss='softmax', wordNgrams=1, dim=150, thread=2, verbose=100)

Read 8M words
Number of words:  140417
Number of labels: 2
Progress: 100.0% words/sec/thread: 1267976 lr:  0.000000 avg.loss:  0.093132 ETA:   0h 0m 0s


In [38]:
# оценим F1 на валидационной выборке.
valid['pred'] = valid["lemma_text"].apply(lambda x: model_ft.predict(str(x), k = 1)[0][0])
valid['pred'] = valid['pred'].apply(lambda x:' '.join((re.sub(r'[\D]',' ',x).split()))).astype(int)
var = f1_score(valid['true'], valid['pred'])
print('F1 = {}'.format(var))

F1 = 0.7975206611570247


**Оценка**:

Эта модель справилась чуть лучше. Однако, главное ее преимущество по сравнению с TF-IDF - лучше справляется с проблемой словарного запаса. Если нет времени и ресурсов на обучение более тяжелых моделий (типа BERT) можно взять ее. Теперь же попробуем добиться более высокого качества при использовании BERT. 

### BERT

На этом шаге я приложу код настройки параметров BERT, как я это делал на удаленном сервере. Также прикладываю [ссылку](https://disk.yandex.ru/d/JDzXq9N_Rv9mDg) с данными обученной модели (конфиг, бины). При желании и возможности использовать GPU можно перевести ячейку с кодом из MarkDown и выполнить обучение. Я сделал 8 эпох, так как торопился по времени ко сроку сдаче. Обучение заняло у меня 13 часов.

Каждое действие в коде буду сопровождать комментарием, чтобы была ясна логика моих шагов.

```python
# выделим необходимые колонки из лемматизированных данных
train = train_lemm[['lemma_text','toxic']]

# загрузим токенизатор и классификатор из предобученной модели BERT
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForSequenceClassification.from_pretrained('bert-base-uncased',num_labels=2)

# переключим модель на ресурсы графического процессора
model = model.to('cuda')

# обозначим таргеты и признаки. Выделим валидационную выборку.
X = list(train["lemma_text"].astype(str))
y = list(train["toxic"].astype(int))
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2,stratify=y)

# токенизируем наши тексты
X_train_tokenized = tokenizer(X_train, padding=True, truncation=True, max_length=512)
X_val_tokenized = tokenizer(X_val, padding=True, truncation=True, max_length=512)

# создадим torch-датасет
class Dataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels=None):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        if self.labels:
            item["labels"] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.encodings["input_ids"])
    
train_dataset = Dataset(X_train_tokenized, y_train)
val_dataset = Dataset(X_val_tokenized, y_val)

# напишем функцию вычисления метрик, которую в последующем будем добавлять в Trainer
def compute_metrics(p):
    print(type(p))
    pred, labels = p
    pred = np.argmax(pred, axis=1)

    accuracy = accuracy_score(y_true=labels, y_pred=pred)
    recall = recall_score(y_true=labels, y_pred=pred)
    precision = precision_score(y_true=labels, y_pred=pred)
    f1 = f1_score(y_true=labels, y_pred=pred)

    return {"accuracy": accuracy, "precision": precision, "recall": recall, "f1": f1}

# определяем аргументы и тренер
args = TrainingArguments(
    output_dir="output",
    num_train_epochs=8,
    per_device_train_batch_size=10)

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics)

# начинаем процесс обучения
trainer.train()

>>> TrainOutput(global_step=81560, training_loss=0.2589868332019561, metrics={'train_runtime': 46937.8872, 'train_samples_per_second': 17.375, 'train_steps_per_second': 1.738, 'total_flos': 2.1458495719784448e+17, 'train_loss': 0.2589868332019561, 'epoch': 8.0})



# здесь можно оценить качество на нашей валидационной выборке
trainer.evaluate()

>>> 
{'eval_loss': 0.16832734644412994,
 'eval_accuracy': 0.95868481971201,
 'eval_precision': 0.8102543399273314,
 'eval_recall': 0.7749034749034749,
 'eval_f1': 0.7921847246891651,
 'eval_runtime': 450.3973,
 'eval_samples_per_second': 56.588,
 'eval_steps_per_second': 7.074,
 'epoch': 8.0}


# а здесь пример, как нам выводить предсказанные значения нашей модели
text = "That was very good"
# создаем инпут, токенизируя текст, полученный для классификации
inputs = tokenizer(text,padding = True, truncation = True, return_tensors='pt').to('cuda')
outputs = model(**inputs)
print(outputs)
# в принте пока что получаем логит, поэтому применяем софтмакс для наших значений
predictions = torch.nn.functional.softmax(outputs.logits, dim=-1)
print(predictions)
# переключаем на cpu и переводим в array
predictions = predictions.cpu().detach().numpy()
predictions

>>> 
SequenceClassifierOutput(loss=None, logits=tensor([[ 2.7286, -3.4281]], device='cuda:0', grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)
tensor([[0.9979, 0.0021]], device='cuda:0', grad_fn=<SoftmaxBackward0>)
array([[0.9978854 , 0.00211461]], dtype=float32)



# сохраним нашу модель
trainer.save_model('CustomModel')


```

Что ж , также добились F1 равный 79. Читал, что если увеличить количество эпох, то можно было сделать побольше. Рекомендуют 10 - 20 эпох. Однако, время сдачи поджимает.

Но на просторах HuggingFace нашел модель, уже обученную для сентимент-анализа. Их там много, но эта встраивается проще всего. Поэтому, можем добавить ее в нашу работу и оценить результат:

In [39]:
tokenizer = BertTokenizer.from_pretrained("JungleLee/bert-toxic-comment-classification")
model = BertForSequenceClassification.from_pretrained("JungleLee/bert-toxic-comment-classification", num_labels=2)

pipeline = TextClassificationPipeline(model=model, tokenizer=tokenizer)

Так как на наших данных мы ее не обучаем, для оценки промежуточного результата я просто добавлю данные из трейн-выборки. но добавлю рандомные 1000 строк, чтобы сократить время предсказания (у меня без GPU на всех данных заняло бы 13 часов)

In [49]:
train_sentiment_bert = train[['lemma_text','true']].sample(1000,random_state=42).reset_index(drop=True)

In [50]:
train_sentiment_bert['true'].value_counts()

true
0    893
1    107
Name: count, dtype: int64

In [42]:
y_pred = []
for text in tqdm(train_sentiment_bert['lemma_text']):
    pred = pipeline(text[:512])
    y_pred.append(1 if pred[0]['label'] == 'toxic' else 0)

100%|███████████████████████████████████████| 1000/1000 [07:14<00:00,  2.30it/s]


In [43]:
train_sentiment_bert['pred'] = pd.Series(y_pred)

In [44]:
var = f1_score(train_sentiment_bert['true'], train_sentiment_bert['pred'])
print('F1 = {}'.format(var))

F1 = 0.7857142857142856


Здесь получили чуть хуже результат, чем в нашей обученной BERT.

## Проверка на тестовой выборке

Для теста выберу модель fastText, чтобы была возможность запустить ячейку и проверить, действительно ли работа выполнена надлежащим образом. Тем более, все модели прошли порог, выставленный заказчиком.

In [45]:
# Если можно, загружаем файл с лемматизированным текстом
# Если файла нет, приведем к лемме

try:
    test = pd.read_csv('test_with_lemm.csv', encoding='utf-8')
except:
    text_raw = test['lemma_text'].astype(str).values
    lemmed_final = []
    nlp = spacy.load('en_core_web_sm')
    for text in nlp.pipe(text_raw,  disable=['parser','ner'], n_process=-1):
        lemmed_final.append(' '.join([token.lemma_ for token in text]))
    test['lemma_text'] = pd.Series(lemmed_final)
    test.to_csv('test_with_lemm.csv',index=False)

In [46]:
# с помощью ранее написанной функции преобразуем таблицу в формат нашей модели
test = create_data_ft(test)

In [47]:
test.head(5)

,toxic,lemma_text,true,toxic_lemma_text
0,__class__1,your mum be a big fat whore your mum be a big ...,1,__class__1 your mum be a big fat whore your mu...
1,__class__0,I have restore the information arbitrarily del...,0,__class__0 I have restore the information arbi...
2,__class__0,vandalism please stop if you continue to vanda...,0,__class__0 vandalism please stop if you contin...
3,__class__0,language some noteworthy and quite convincing ...,0,__class__0 language some noteworthy and quite ...
4,__class__0,please feel free to add anything you feel I 'v...,0,__class__0 please feel free to add anything yo...


In [48]:
# оценим F1 на валидационной выборке.
test['pred'] = test["lemma_text"].apply(lambda x: model_ft.predict(str(x), k = 1)[0][0])
test['pred'] = test['pred'].apply(lambda x:' '.join((re.sub(r'[\D]',' ',x).split()))).astype(int)
var = f1_score(test['true'], test['pred'])
print('Test F1 = {}'.format(var))

Test F1 = 0.7998681173755356


## Общий вывод по работе

В начале нашей работы мы импортировали все библиотеки и изучили исходные данные. Из предобработки этих данных было выполнено:

- очистка от ненужных символов с помощью регулярных выражений;

- лемматизация с использованием библиотеки spacy;

- разделение на тестовую и тренировочную выборки.

Далее план работы с моделями был определен следующий:

- обучить модель на основе использования TF-IDF

- обучить модель с помощью одной из библиотек векторного представления языка (Word2Vec, fastText)

- обучить модель BERT.


На первом шаге векторизировали наши данные. При этом для признаков использовали векторы слов и векторы n-грамм вместе. После обучили модель логистической регрессии.

На втором шаге подготовили данные для библиотеки fastText и обучили модель, оценив качество на валидационной выборке.

На третьем этапе обучали BERT с использованием предобученной базовой модели. Обучение проходило на сторонней машине, использовался графический процессор, выбрано было 8 эпох. Также попробовали взять уже обученную модель для сентимент-анализа JungleLee, но результат был хуже, чем у нашей модели.

Теперь можно отметить качество по заранее определенной метрике(F1).

Все наши модели выдали качество на валидационной выборке 0,79.

При этом стоит заметить, что TF-IDF будет проигрывать в перспективном использовании, так как не учитывает проблему словарного запаса.

Модель BERT - хороший вариант, но тонкая настройка занимает много времени и требует GPU. Показатель качества еще можно поднять, за счет увеличения эпох.

fastText - решает проблему времени обучения и словарного запаса. Для себя сделал вывод, что отлично подходит для базовой модели. Если надо быстро нащупать какой-то результат - отличный выбор. Для длительной работы обучал бы BERT.

На тестовой выборке был испытан fastText. F1 score = 0,799